# Python as a BI Tool

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline

## Data Input and Cleaning

### 1) Loan

In [2]:
loan=pd.read_csv('loan1.csv')
loan['status'].replace({"A": "Contract Finished, No Problems", "B": "Contract Finished, Loan Not Paid",
                                'C':'Running Contract, OK so far','D':'Running Contract, Client in Debt'}, inplace=True)
loan['Loan_date'] = pd.to_datetime(loan['Modified_date'].astype(str), format='%Y%m%d')
loan=loan.drop(['date','Modified_date'],axis=1)
loan.head(3)

,loan_id,account_id,amount,duration,payments,status,Loan_date
0,5314,1787,96396,12,8033,"Contract Finished, Loan Not Paid",1993-07-05
1,5316,1801,165960,36,4610,"Contract Finished, No Problems",1993-07-11
2,6863,9188,127080,60,2118,"Contract Finished, No Problems",1993-07-28


### 2) Client 

In [3]:
client=pd.read_csv('client1.csv')
client['birth_date'] = pd.to_datetime(client['birth_date'].astype(str), format='%Y%m%d')
client=client.drop(['birth_number','corrected_birthnum'],axis=1)
client['Age']= 1998-client['birth_date'].apply(lambda i: i.year)
client.head(3)

,client_id,district_id,gender,birth_date,Age
0,1,18,F,1970-12-13,28
1,2,1,M,1945-02-04,53
2,3,1,F,1940-10-09,58


### 3) District

In [4]:
district=pd.read_csv('district.csv')
district=district.drop(['A5','A6','A7','A8','A9','A10','A12','A13','A14','A15','A16'],axis=1)
district=district.rename(columns={'A1':'district_id','A2':'District_Name','A3':'Region','A4':'Population','A11':'Avg_Salary'})
district.head(3)

,district_id,District_Name,Region,Population,Avg_Salary
0,1,Hl.m. Praha,Prague,1204953,12541
1,2,Benesov,central Bohemia,88884,8507
2,3,Beroun,central Bohemia,75232,8980


### 4) Account

In [5]:
account=pd.read_csv('account1.csv')
account['frequency'].replace({"POPLATEK MESICNE": "Monthly Issuance", "POPLATEK TYDNE": "Weekly Issuance",
                             'POPLATEK PO OBRATU':'Issuance After Transaction'}, inplace=True)
account['Account_date'] = pd.to_datetime(account['Modified_date'].astype(str), format='%Y%m%d')
account=account.drop(['date','Modified_date'],axis=1)
account.head(3)

,account_id,district_id,frequency,Account_date
0,576,55,Monthly Issuance,1993-01-01
1,3818,74,Monthly Issuance,1993-01-01
2,704,55,Monthly Issuance,1993-01-01


### 5) Disposition

In [6]:
disp=pd.read_csv('disp.csv')
disp=disp.rename(columns={'type':'Disp_type'})
disp.head(3)

,disp_id,client_id,account_id,Disp_type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT


### 6) Order

In [7]:
order=pd.read_csv('order1.csv')
order['k_symbol'].replace({"POJISTNE": "Insurance Payment", "SIPO": "Household Payment",'LEASING':'Leasing',
                             'UVER':'Loan Payment',' ':'Other Payments'}, inplace=True)
order=order.rename(columns={'k_symbol':'Order_Description'})
order=order.drop(['account_to'],axis=1)
order.head(3)

,order_id,account_id,bank_to,amount,Order_Description
0,29401,1,YZ,2452.0,Household Payment
1,29402,2,ST,3372.7,Loan Payment
2,29403,2,QR,7266.0,Household Payment


### 7) Card

In [8]:
card=pd.read_csv('card1.csv')
card['Card_issued'] = pd.to_datetime(card['issued_date'].astype(str), format='%Y%m%d')
card=card.drop(['issued','issued_date'],axis=1)
card=card.rename(columns={'type':'Card_type'})
card.head(3)

,card_id,disp_id,Card_type,Card_issued
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05


### 8) Transaction

In [9]:
trans=pd.read_csv('trans1.csv')
trans['type'].replace({"PRIJEM": "Credit", "VYDAJ": "Withdrawal"},inplace=True)
trans['operation'].replace({"VYBER KARTOU": "Credit Card Withdrawal", "VKLAD": "Credit in Cash",
                            'PREVOD Z UCTU':'Collection from Another Bank', "VYBER":"Withdrawal in Cash",
                             'PREVOD NA UCET':'Remittance to Another Bank'}, inplace=True)
trans['operation'].fillna('Other transactions',inplace=True)
trans=trans.drop(['delete','balance','account','bank','k_symbol'],axis=1)
trans=trans.rename(columns={'date':'Trans_date','type':'trans_type','amount':'trans_amount'})
trans['Trans_date']=pd.to_datetime(trans['Trans_date'])
trans.head(3)

,trans_id,account_id,Trans_date,trans_type,operation,trans_amount
0,3548438,537,1998-07-31,Credit,Other transactions,208.9
1,841814,2865,1995-10-11,Withdrawal,Remittance to Another Bank,1.0
2,121479,408,1997-10-06,Credit,Credit in Cash,13304.0


## Generating PPTs having visualizations

### Viz-1: Simple Column Chart

In [10]:
#Creating presentation, defining layout and creating a new slide
from pptx import Presentation
prs=Presentation()
layout=prs.slide_layouts[5]
slide=prs.slides.add_slide(layout)

#Importing libraries to plot the chart
from pptx.util import Inches
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE

#Merging Client and District tables for our chart
table=pd.merge(client,district,how='inner',on='district_id')

#Specifying the data for categories(dimension) and series(values)
mydata=CategoryChartData()
x=table.groupby('District_Name').count()['client_id'].sort_values(ascending=False).iloc[0:10]
mydata.categories=x.index
mydata.add_series('No. of clients',x.values)

#Specifying Left, Top, Width and Height margins
x,y,cx,cy=Inches(2),Inches(2),Inches(8),Inches(5.5)

#Creating chart based on chart type and data
chart=slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED,x,y,cx,cy,mydata).chart

#Adding legend and title
chart.has_legend=True
from pptx.util import Pt
chart.chart_title.has_text_frame=True
chart.chart_title.text_frame.text='Top 10 Districts with most clients'
chart.chart_title.text_frame.paragraphs[0].font.size = Pt(20)
chart.chart_title.text_frame.paragraphs[0].font.bold = True

#Saving as ppt
prs.save('Chart1.pptx')

### Viz-2: Stacked Bar Chart

In [11]:
#Grouping by bank and description in ORDER table
grp=order.groupby(['bank_to','Order_Description'])['amount'].sum()
grp

bank_to  Order_Description
AB       Household Payment    1111015.0
         Insurance Payment      79952.0
         Leasing                57444.0
         Loan Payment          238914.5
         Other Payments        220064.0
                                ...    
YZ       Household Payment    1072609.0
         Insurance Payment      72230.0
         Leasing                72758.4
         Loan Payment          228437.4
         Other Payments        190948.0
Name: amount, Length: 65, dtype: float64

In [12]:
slide=prs.slides.add_slide(layout)
mydata=CategoryChartData()

#Inputing various banks as dimension
mydata.categories=np.sort(order['bank_to'].unique())

#Inputing order description amounts as metric via loop
for i in order['Order_Description'].unique():
    mydata.add_series(i,grp[grp.index.get_level_values('Order_Description') == i].values)
    
x,y,cx,cy=Inches(2),Inches(2),Inches(8),Inches(5.5)
chart=slide.shapes.add_chart(XL_CHART_TYPE.BAR_STACKED,x,y,cx,cy,mydata).chart

chart.has_legend=True
from pptx.util import Pt
chart.chart_title.has_text_frame=True
chart.chart_title.text_frame.text='Total Payment Amount(USD) by Type for Each Bank'
chart.chart_title.text_frame.paragraphs[0].font.size = Pt(20)
chart.chart_title.text_frame.paragraphs[0].font.bold = True

prs.save('Chart2.pptx')

### Viz-3: Time Series (Line chart)

In [13]:
trans.set_index (trans['Trans_date'], inplace = True)
trans=trans.sort_index()

#Adding yearly inflow and outflow columns to TRANS table
trans= trans.assign (positive = np.nan)
trans= trans.assign (negative= np.nan)

trans['positive'] = trans['trans_amount'].where (trans['operation'] == 'Credit in Cash', trans['positive'] )
trans['positive'] = trans['trans_amount'].where (trans['operation'] == 'Collection from Another Bank', trans['positive'])

trans['negative'] = trans['trans_amount'].where (trans['operation'] == 'Withdrawal in Cash', trans['negative'] )
trans['negative'] = trans['trans_amount'].where (trans['operation'] == 'Remittance to Another Bank', trans['negative'] )
trans['negative'] = trans['trans_amount'].where (trans['operation'] == 'Credit Card Withdrawal', trans['negative'])

inflowsY = trans['positive'].resample('Y').sum ()
outflowsY = trans['negative'].resample('Y').sum ()

#Separating date on basis of year and month_year
trans['Year']=trans['Trans_date'].apply(lambda i:i.year)
trans['month_year'] = trans['Trans_date'].apply(lambda x: x.strftime('%b-%y')) 

slide=prs.slides.add_slide(layout)
mydata=CategoryChartData()

#Inputing Years as dimension
mydata.categories=np.sort(trans['Year'].unique())

#Inputing Total inflow and outflow amount as dimension
mydata.add_series('Fund Inflow',inflowsY.values/1000000)
mydata.add_series('Fund Outflow',outflowsY.values/1000000)
    
x,y,cx,cy=Inches(2),Inches(2),Inches(8),Inches(5.5)
chart=slide.shapes.add_chart(XL_CHART_TYPE.LINE,x,y,cx,cy,mydata).chart

chart.has_legend=True
from pptx.util import Pt
chart.chart_title.has_text_frame=True
chart.chart_title.text_frame.text='Banks Yearly Fund Flow (Millions USD)'
chart.chart_title.text_frame.paragraphs[0].font.size = Pt(20)
chart.chart_title.text_frame.paragraphs[0].font.bold = True

prs.save('Chart3.pptx')

In [14]:
#Adding monthly inflow and outflow columns to TRANS table
inflowsM = trans['positive'].resample('M').sum ()
outflowsM = trans['negative'].resample('M').sum ()

slide=prs.slides.add_slide(layout)
mydata=CategoryChartData()

#Inputing Years as dimension
mydata.categories=sorted(trans['month_year'].unique().tolist(),key=lambda date:datetime.strptime(date,'%b-%y'))

#Inputing Total inflow and outflow amount as dimension
mydata.add_series('Fund Inflow',inflowsM.values/1000000)
mydata.add_series('Fund Outflow',outflowsM.values/1000000)
    
x,y,cx,cy=Inches(2),Inches(2),Inches(8),Inches(5.5)
chart=slide.shapes.add_chart(XL_CHART_TYPE.LINE,x,y,cx,cy,mydata).chart

chart.has_legend=True
from pptx.util import Pt
chart.chart_title.has_text_frame=True
chart.chart_title.text_frame.text='Banks Monthly Fund Flow (Millions USD)'
chart.chart_title.text_frame.paragraphs[0].font.size = Pt(20)
chart.chart_title.text_frame.paragraphs[0].font.bold = True

prs.save('Chart4.pptx')

### Viz-4: XY Scatter Plot 

In [15]:
slide=prs.slides.add_slide(layout)
from pptx.chart.data import XySeriesData, XyChartData
mydata = XyChartData()

#Adding x and y data points
x_values_list=loan.groupby('status').mean()['amount'].values
y_values_list=loan.groupby('status').mean()['payments'].values

md=mydata.add_series('Loan Status', number_format= None)
for x, y in list(zip(x_values_list, y_values_list)):
         md.add_data_point(x, y, number_format=None)

x,y,cx,cy=Inches(2),Inches(2),Inches(8),Inches(5.5)
chart = slide.shapes.add_chart(XL_CHART_TYPE.XY_SCATTER, x, y, cx, cy, mydata).chart

chart.has_legend=True
from pptx.util import Pt
chart.chart_title.has_text_frame=True
chart.chart_title.text_frame.text='Correlation of default with loan amounts and payments'
chart.chart_title.text_frame.paragraphs[0].font.size = Pt(20)
chart.chart_title.text_frame.paragraphs[0].font.bold = True

#Adding axes titles
category_axis_title = chart.category_axis.axis_title
category_axis_title.text_frame.text = "Avg Loan Amount"
value_axis_title = chart.value_axis.axis_title
value_axis_title.text_frame.text = "Avg Monthly Payment"
category_axis = chart.category_axis
category_axis.has_major_gridlines=True
value_axis=chart.value_axis
value_axis.minimum_scale = 2000

prs.save('Chart5.pptx')

### Multiple Charts in one slide

In [16]:
slide=prs.slides.add_slide(layout)

#Specifying the data for categories(dimension) and series(values)
mydata=CategoryChartData()
x=table.groupby('District_Name').count()['client_id'].sort_values(ascending=False).iloc[0:10]
mydata.categories=x.index
mydata.add_series('No. of clients',x.values)

mydata2=CategoryChartData()
x=table.groupby('Region').count()['client_id'].sort_values(ascending=False)
mydata2.categories=x.index
mydata2.add_series('No. of clients',x.values)

#Specifying Left, Top, Width and Height margins
x,y,cx,cy=Inches(0),Inches(2),Inches(5),Inches(5)
x2,y2,cx2,cy2=Inches(5),Inches(2),Inches(5),Inches(5)

#Creating chart based on chart type and data
chart=slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED,x,y,cx,cy,mydata).chart
chart2=slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED,x2,y2,cx2,cy2,mydata2).chart

#Adding legend and title
from pptx.util import Pt
chart.chart_title.has_text_frame=True
chart.chart_title.text_frame.text='Top 10 Districts with most clients'
chart.chart_title.text_frame.paragraphs[0].font.size = Pt(20)
chart.chart_title.text_frame.paragraphs[0].font.bold = True

chart2.chart_title.has_text_frame=True
chart2.chart_title.text_frame.text='Regions with most clients'
chart2.chart_title.text_frame.paragraphs[0].font.size = Pt(20)
chart2.chart_title.text_frame.paragraphs[0].font.bold = True

#Saving as ppt
prs.save('Chart6.pptx')

### Adding a Table

In [19]:
slide=prs.slides.add_slide(layout)

table_placeholder=slide.shapes[0]

#Specifying Left, Top, Width and Height margins
x,y,cx,cy=Inches(0),Inches(6),Inches(10),Inches(1)

#Creating chart based on chart type and data
shape=slide.shapes.add_table(2,11,x,y,cx,cy)
table.first_col=True

table=shape.table

joint=pd.merge(client,district,how='inner',on='district_id')
joint1=joint.groupby('Region').count()['client_id'].sort_values(ascending=False).iloc[0:10]

#Adding legend and title

cell1=table.cell(0,0)
cell1.text='Region'
cell2=table.cell(1,0)
cell2.text='Population'
from pptx.dml.color import RGBColor
cell1.fill.solid()
cell1.fill.fore_color.rgb = RGBColor(185, 70, 70)
cell2.fill.solid()
cell2.fill.fore_color.rgb = RGBColor(235,224,224)

cnt=1
for i in joint1.index:
    cell = table.cell(0, cnt)
    cell.text = i
    cnt=cnt+1
cnt=1
for i in joint1.index:
    cell = table.cell(1, cnt)
    cell.text = str(round((district[district['Region']==i]['Population'].mean()/1000)))+'K'
    cnt=cnt+1

#Saving as ppt
prs.save('Table1.pptx')